In [ ]:
!pip install kay

In [1]:
from langchain.chat_models import ChatAnthropic
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser

In [7]:
prompt = ChatPromptTemplate.from_messages([
    ("system", """You are an expert at world knowledge. Your task is to step back and paraphrase a question to a more generic step-back question, which is easier to answer. Here are a few examples:"""),
    ("user", "Could the members of The Police perform lawful arrests?"),
    ("assistant", "what can the members of The Police do?"),
    ("user", "Jan Sindel’s was born in what country?"),
    ("assistant", "what is Jan Sindel’s personal history?"),
    ("user", "{question}"),
])

In [8]:
question_gen = prompt | ChatAnthropic() | StrOutputParser()

In [9]:
question_gen.invoke({"question":"how is langchain different than langsmith?"})

' What are the differences between langchain and langsmith?'

  Obtaining dependency information for kay from https://files.pythonhosted.org/packages/0a/98/a26cb7250bec7735383434d15a377a0b6f2fe385b2a0f5024d26c871ab3e/kay-0.1.2-py3-none-any.whl.metadata


In [29]:
from langchain.retrievers import KayAiRetriever

retriever = KayAiRetriever.create(dataset_id="company", data_types=["10-K", "10-Q"], num_contexts=6)


In [30]:
response_prompt_template = """You are an expert of world knowledge. I am going to ask you a question. Your response should be comprehensive and not contradicted with the following context if they are relevant. Otherwise, ignore them if they are not relevant.

{normal_context}
{step_back_context}

Original Question: {question}
Answer:"""
response_prompt = ChatPromptTemplate.from_template(response_prompt_template)

In [31]:
chain = {
    "normal_context": (lambda x: x['question']) | retriver | (lambda x: x[:3]),
    "step_back_context": question_gen | retriver | (lambda x: x[:3]),
    "question": lambda x: x["question"]
} | response_prompt | ChatAnthropic() | StrOutputParser()

In [33]:
chain.invoke({"question": "who has more revenue etsy or pinterest?"})

" Based on the provided context documents and information, it appears that Etsy generates significantly more revenue than Pinterest. The key points are:\n\n- Etsy generated $2.5 billion in revenue in 2022, while Pinterest generated $2.58 billion in revenue in 2022. \n\n- Etsy is an e-commerce marketplace that generates revenue through seller fees and commissions on transactions. Pinterest is an image sharing and discovery platform that generates revenue primarily through advertising.\n\n- Etsy has experienced strong revenue growth in recent years, increasing 10.1% year-over-year in 2022. Pinterest's revenue grew 8% year-over-year in 2022.\n\n- Etsy has over 7 million active sellers on its marketplace platform, while Pinterest does not facilitate transactions or have sellers. \n\n- The business models and revenue sources are very different between the two companies, but the available revenue numbers indicate Etsy generates more total revenue than Pinterest.\n\nIn summary, the data indic